# Step 1: Data Extraction

## Purpose
The purpose of this notebook is to download the Open Food Facts dataset ([food.parquet on Hugging Face](https://huggingface.co/datasets/openfoodfacts/product-database/blob/main/food.parquet)), perform an initial data inspection, and extract a relevant subset tailored for this project.

The raw dataset contains nearly 4 million rows and 110 columns, including several semi-structured JSON fields that require parsing and transformation. We need understanding the structure, cleaning, and selecting the essential features needed for subsequent EDA and preprocessing steps.


---

## Tasks
- Connect to Hugging Face and download the raw dataset.
- Inspect dataset structure: review number of rows, columns, datatypes, and memory usage.
- Filter and clean the dataset:
    - Filter rows based on project-specific conditions.
    - Select relevant columns required for analysis.
    - Parse JSON fields and flatten nested structures.
    - Format and rename columns for clarity and consistency.
- Save the extracted dataset

---

## Output
- `food.csv` — a filtered and structured dataset ready for preprocessing and EDA.


In [28]:
import os
import requests
import json
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

#### 1. Download the dataset from Hugging Face

In [29]:
url = "https://huggingface.co/datasets/openfoodfacts/product-database/resolve/main/food.parquet"
dataset_path = "../data/food.parquet"

if os.path.exists(dataset_path):
    print(f"Dataset '{dataset_path}' already exists locally. Skipping download.\n")

else:
    print(f"Downloading '{dataset_path}' from Hugging Face...")

    response = requests.get(url, stream=True)
    response.raise_for_status()

    with open(dataset_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

    print(f"Dataset successfully downloaded and saved to '{dataset_path}'.\n")


Dataset '../data/food.parquet' already exists locally. Skipping download.



#### 2. Inspect dataset

In [30]:
# Inspect the first 100 rows
parquet_file = pq.ParquetFile(dataset_path)
batch = next(parquet_file.iter_batches(batch_size=100, columns=None))
df_first_100 = batch.to_pandas()
print(df_first_100.shape)
print(df_first_100.dtypes)
df_first_100.head()

(100, 110)
additives_n                      float64
additives_tags                    object
allergens_tags                    object
brands_tags                       object
brands                            object
                                  ...   
unknown_ingredients_n            float64
unknown_nutrients_tags            object
vitamins_tags                     object
with_non_nutritive_sweeteners    float64
with_sweeteners                  float64
Length: 110, dtype: object


,additives_n,additives_tags,allergens_tags,brands_tags,brands,categories,categories_tags,categories_properties,checkers_tags,ciqual_food_name_tags,...,states_tags,stores_tags,stores,traces_tags,unique_scans_n,unknown_ingredients_n,unknown_nutrients_tags,vitamins_tags,with_non_nutritive_sweeteners,with_sweeteners
0,NaN,None,[en:nuts],[xx:bovetti],Bovetti,"Petit-déjeuners,Produits à tartiner,Produits à...","[en:breakfasts, en:spreads, en:sweet-spreads, ...","{'ciqual_food_code': 31032.0, 'agribalyse_food...",[],[chocolate-spread-with-hazelnuts],...,"[en:to-be-completed, en:nutrition-facts-comple...",[],,[],1.0,NaN,[],[],NaN,NaN
1,0.0,[],[],[lagg-s],Lagg's,null,[en:null],"{'ciqual_food_code': None, 'agribalyse_food_co...",[],[unknown],...,"[en:to-be-completed, en:nutrition-facts-comple...",None,None,[],1.0,0.0,[],[],NaN,NaN
2,0.0,[],[],[lagg-s],Lagg's,"Plant-based foods and beverages, Beverages, Ho...","[en:plant-based-foods-and-beverages, en:bevera...","{'ciqual_food_code': 18020.0, 'agribalyse_food...",[],[unknown],...,"[en:to-be-completed, en:nutrition-facts-comple...",None,None,[],NaN,0.0,[],[],NaN,NaN
3,0.0,[],[],[xx:lagg-s],Lagg's,"Beverages and beverages preparations, Plant-ba...","[en:beverages-and-beverages-preparations, en:p...","{'ciqual_food_code': 18020.0, 'agribalyse_food...",[],[unknown],...,"[en:to-be-completed, en:nutrition-facts-comple...",None,None,[],NaN,1.0,[],[],NaN,NaN
4,0.0,[],[],[lagg-s],Lagg's,None,None,"{'ciqual_food_code': None, 'agribalyse_food_co...",[],[unknown],...,"[en:to-be-completed, en:nutrition-facts-comple...",None,None,[],NaN,0.0,[],[],NaN,NaN


In [31]:
# Save the first 100 rows to a JSON file
df_first_100.to_json("../data/raw_dataset_subset.json", orient="records", lines=True)

#### 3. Filter and clean the dataset

In [32]:
# Helper functions to parse / clean specific columns

def parse_tags(tags):
  if isinstance(tags, (np.ndarray, list, tuple, pd.Series)):
      if len(tags) == 0:
          return []
      tags = [str(tag) for tag in tags if tag is not None]
  else:
      return []
  return [tag.split(":")[-1] for tag in tags if tag.startswith("en:") and tag.split(":")[-1] != 'null']


def parse_ingredients(ing_str):
  try:
      ings = json.loads(ing_str) if isinstance(ing_str, str) else ing_str
      return [item.get("text").lower() for item in ings] if isinstance(ings, list) else []
  except:
      return []
  

def extract_nutriments(nutriments_list):
  if not isinstance(nutriments_list, (np.ndarray, list, tuple, pd.Series)):
      return pd.Series()

  nutri_dict = {item['name'].replace('-', '_'): item.get('100g') or item.get('value') for item in nutriments_list
                if item['name'] in ["energy", "sugars", "added-sugars", "carbohydrates", "salt", "fat", "trans-fat", "proteins"]}
  return pd.Series(nutri_dict)

In [33]:
output_dataset = "../data/food.csv"

# Remove output file if it exists
if os.path.exists(output_dataset):
    os.remove(output_dataset)

required_columns = [
    "code", "brands", "product_name", "categories_tags", "food_groups_tags", "labels_tags",
    "ingredients_n", "ingredients_from_palm_oil_n", "ingredients", "nutriments",
    "nutriscore_grade", "lang", "additives_n", "additives_tags", "allergens_tags",
    "ingredients_analysis_tags", "completeness"
]

parquet_file = pq.ParquetFile(dataset_path)
batch_size = 10000
i = 0

print(f"Cleaning dataset in batches (size = {batch_size})...")

for batch in parquet_file.iter_batches(batch_size=batch_size, columns=required_columns):
    print(f"Processing batch {i+1} (rows {batch_size*i + 1}-{batch_size*(i+1)})...")

    df = batch.to_pandas()

    # Filter out invalid or blank nutriscore grade
    df = df[df["nutriscore_grade"].isin(["a", "b", "c", "d", "e"])]

    # Filter out products with no ingredients
    df = df[~df["ingredients"].isna()]

    # Filter for English language products
    df = df[df["lang"] == "en"]

    # Skip empty batches
    if df.empty:
        i += 1
        continue

    # Get product name
    df["product"] = df["product_name"].apply(lambda x: x[0]["text"] if isinstance(x, np.ndarray) and len(x) > 0 and "text" in x[0] else x)
    
    # Parse ingredients
    df["ingredients"] = df["ingredients"].apply(parse_ingredients)

    # Parse nutriments
    nutri_extracted = df["nutriments"].apply(extract_nutriments)
    for col in nutri_extracted.columns:
        df[col] = nutri_extracted[col]
    df.drop(columns=["nutriments"], inplace=True)

    # Clean tags
    for c in df.columns:
        if c.endswith("_tags"):
            df[c] = df[c].apply(lambda x: [item.replace("en:", "") for item in x] if isinstance(x, np.ndarray) else str(x))

    # Reorder and ensure all final columns exist
    final_cols = [
        "code", "brands", "product", "lang", "categories_tags", "food_groups_tags", "labels_tags",
        "additives_n", "additives_tags", "allergens_tags",
        "ingredients_analysis_tags", "ingredients_n", "ingredients_from_palm_oil_n", "ingredients",
        "completeness", "energy", "sugars", "added_sugars", "carbohydrates", "salt", "fat",
        "trans_fat", "proteins", "nutriscore_grade"
    ]
    for col in final_cols:
        if col not in df:
            df[col] = None
    df = df[final_cols]

    # Write batch to CSV incrementally
    if i == 0:
        header_written = False
    else:
        header_written = True

    df.to_csv(output_dataset, mode='a', header=not header_written, index=False)

    # Free memory
    del df, nutri_extracted, batch
    import gc
    gc.collect()

    i += 1

print(f"\nProcessing complete. Output saved to {output_dataset}.")

Cleaning dataset in batches (size = 10000)...
Processing batch 1 (rows 1-10000)...
Processing batch 2 (rows 10001-20000)...
Processing batch 3 (rows 20001-30000)...
Processing batch 4 (rows 30001-40000)...
Processing batch 5 (rows 40001-50000)...
Processing batch 6 (rows 50001-60000)...
Processing batch 7 (rows 60001-70000)...
Processing batch 8 (rows 70001-80000)...
Processing batch 9 (rows 80001-90000)...
Processing batch 10 (rows 90001-100000)...
Processing batch 11 (rows 100001-110000)...
Processing batch 12 (rows 110001-120000)...
Processing batch 13 (rows 120001-130000)...
Processing batch 14 (rows 130001-140000)...
Processing batch 15 (rows 140001-150000)...
Processing batch 16 (rows 150001-160000)...
Processing batch 17 (rows 160001-170000)...
Processing batch 18 (rows 170001-180000)...
Processing batch 19 (rows 180001-190000)...
Processing batch 20 (rows 190001-200000)...
Processing batch 21 (rows 200001-210000)...
Processing batch 22 (rows 210001-220000)...
Processing batch 2

In [34]:
# Read and display the cleaned dataset
df = pd.read_csv(output_dataset)
print(df.shape)
df.head()

# Save the first 100 rows of the cleaned dataset to a JSON file
df.head(100).to_json("../data/cleaned_dataset_subset.json", orient="records", lines=True)

(424297, 24)


/var/folders/p3/dqqbfzkd0ln207r8z90_jbnc0000gp/T/ipykernel_57773/1698039606.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(output_dataset)
